In [ ]:
from garminconnect import Garmin
import matplotlib.pyplot as plt
import pandas as pd

username = "janicksteffen@hotmail.com"
password = "07@Janick@98"

client = Garmin(username, password)
client.login()

activities = client.get_activities(0, 30)

activity_data = {
    "Date": [],
    "Steps": [],
    "Calories": [],
    "Distance_km": [],
    "Avg_HeartRate": [],
}

for activity in activities:
    activity_data["Date"].append(activity["startTimeLocal"])
    activity_data["Steps"].append(activity.get("steps", 0))
    activity_data["Calories"].append(activity.get("calories", 0))
    activity_data["Distance_km"].append(activity.get("distance", 0) / 1000)
    activity_data["Avg_HeartRate"].append(activity.get("averageHR", 0))

df = pd.DataFrame(activity_data)

plt.plot(df["Date"], df["Steps"], marker="o", linestyle="-")
plt.xlabel("Datum")
plt.ylabel("Schritte")
plt.title("Schritte pro Tag")
plt.xticks(rotation=45)
plt.show()

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Activity Dashboard"),
    dcc.Dropdown(
        id="metric-dropdown",
        options=[
            {"label": "Steps", "value": "Steps"},
            {"label": "Calories", "value": "Calories"},
            {"label": "Distance (km)", "value": "Distance_km"},
            {"label": "Average Heart Rate", "value": "Avg_HeartRate"}
        ],
        value="Steps"
    ),
    dcc.Graph(id="activity-graph")
])

@app.callback(
    Output("activity-graph", "figure"),
    [Input("metric-dropdown", "value")]
)
def update_graph(selected_metric):
    fig = px.line(df, x="Date", y=selected_metric, markers=True, title=f"{selected_metric} per Day")
    fig.update_layout(xaxis_title="Date", yaxis_title=selected_metric)
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)
